# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.17.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set


> Applying preprocessing only to the training set to prevent overfitting and apply data variations through geometric transformations(zoom,rotation,flipping etc.) of image, also called "**Image Augmentation**".



In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255, # Apply feature scalling to each pixel by dividing the value by 255
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64), # Size of the input images; reduced to increase processing speed.
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255) # Only Apply feature scalling to each pixel
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution
#### Conv2D Layer Parameters:


> **filter**-> how many feature detector we want, <br>
  **kernel_size** -> easch feature detector matrix of 3x3 size ,<br>
  **activation** -> activation function used for convolutional layer ( Rectifier function )<br>
  **input_shape** -> input image er size, and '3' for colored image; "1" if black and white image  




In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

C:\Users\User\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


C:\Users\User\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 302s 1s/step - accuracy: 0.5646 - loss: 0.6769 - val_accuracy: 0.7020 - val_loss: 0.5995
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 93s 357ms/step - accuracy: 0.6826 - loss: 0.6067 - val_accuracy: 0.7275 - val_loss: 0.5692
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 231ms/step - accuracy: 0.7198 - loss: 0.5565 - val_accuracy: 0.7300 - val_loss: 0.5425
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 230ms/step - accuracy: 0.7275 - loss: 0.5290 - val_accuracy: 0.7595 - val_loss: 0.5041
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 231ms/step - accuracy: 0.7475 - loss: 0.5069 - val_accuracy: 0.7760 - val_loss: 0.4876
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 230ms/step - accuracy: 0.7672 - loss: 0.4779 - val_accuracy: 0.7725 - val_loss: 0.4834
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 230ms/step - accuracy: 0.7748 - loss: 0.4717 - val_accuracy: 0.7445 - val_loss: 0.5238
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 230ms/step - accuracy: 0.7763 - loss: 0.4556 - val_a

## Part 4 - Making a single prediction

In [37]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_5.jpeg', target_size = (64, 64)) # cat
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


{'cats': 0, 'dogs': 1}

In [38]:
print( 'dog' if result[0][0] == 1 else 'cat' )

cat


In [39]:
test_image = image.load_img('dataset/single_prediction/cat_or_dog_7.jpeg', target_size = (64, 64)) # dog
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print( 'dog' if result[0][0] == 1 else 'cat' )


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
dog


In [41]:
test_image = image.load_img('dataset/single_prediction/cat_or_dog_4.jpg', target_size = (64, 64)) # cat
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print( 'dog' if result[0][0] == 1 else 'cat' )


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
cat


In [40]:
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64)) # dog
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print( 'dog' if result[0][0] == 1 else 'cat' )


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
dog
